# resources

http://www.petercollingridge.co.uk/pygame-3d-graphics-tutorial/rotation-3d

https://www.siggraph.org/education/materials/HyperGraph/modeling/mod_tran/3drota.htm#Z

In [1]:
import bvh
import math
import pyglet
import numpy as np
from pyglet.gl import *
from transforms3d import axangles

Each bvh file has 2 main parts, the skeletal structure part (with information about what joints split off from waht other joints). It also contains at what distance each joint is from its parent joint. Italso has information about what the 'channels' are for those joints. Channels are just values, so for example your channels could be RGB colors (R,G,B, being the channels) or they could be X,Y,Z position, or X,Y,Z rotation. The second part which contains all the frames, which say, 'at frame N, the channels for joint X will be X1,Y1,Z1, and at frame N+1 they will be X2,Y2,Z2, etc.

First lets try to get a list of all joints in the bvh

In [2]:
with open('/Users/yvanscher/Downloads/02_01.bvh') as f:
    test_bvh = bvh.Bvh(f.read())
    for x in test_bvh.get_joints():
        print(x)

ROOT Hips
JOINT LHipJoint
JOINT LeftUpLeg
JOINT LeftLeg
JOINT LeftFoot
JOINT LeftToeBase
JOINT RHipJoint
JOINT RightUpLeg
JOINT RightLeg
JOINT RightFoot
JOINT RightToeBase
JOINT LowerBack
JOINT Spine
JOINT Spine1
JOINT Neck
JOINT Neck1
JOINT Head
JOINT LeftShoulder
JOINT LeftArm
JOINT LeftForeArm
JOINT LeftHand
JOINT LeftFingerBase
JOINT LeftHandIndex1
JOINT LThumb
JOINT RightShoulder
JOINT RightArm
JOINT RightForeArm
JOINT RightHand
JOINT RightFingerBase
JOINT RightHandIndex1
JOINT RThumb


Cool these are all the joints in the file. Let's get the points for just hte root hip joint.

In [3]:
file = '/Users/yvanscher/Downloads/02_01.bvh'

def get_joint_channels(file, joint_name):
    with open(file) as f:
        bvh_r = bvh.Bvh(f.read())
        return bvh_r.joint_channels(joint_name)

# get the animation for nay joint
def get_joint_anim(file, joint_name):
    with open(file) as f:
        bvh_r = bvh.Bvh(f.read())
        channels = bvh_r.joint_channels(joint_name)
        return np.array(bvh_r.frames_joint_channels(joint_name, channels))

# get all but the hip animation
def get_anims(file):
    with open(file) as f:
        bvh_r = bvh.Bvh(f.read())
        joints = [joint for joint in bvh_r.get_joints()]
        anim_joints = np.zeros((bvh_r.nframes, len(joints), 3))
        for i,joint in enumerate(joints):
            if joint.name == 'Hips':
                continue
            channels = bvh_r.joint_channels(joint.name)
            anim = np.array(bvh_r.frames_joint_channels(joint.name, channels))[:,:]
            anim_joints[:,i] = anim
    return anim_joints

def scale_range(a, minv, maxv):
    a -= np.min(a)
    a /= np.max(a) / (maxv - minv)
    a += minv
    return a

hip_anim = get_joint_anim(file, 'Hips')
other_anim = get_anims(file)
assert hip_anim.shape == (344, 6)
assert other_anim.shape == (344, 31, 3)

In [4]:
hip_channels = get_joint_channels(file, 'Hips')
print(f'hip channels: {hip_channels}')
print(f'hip data: {hip_anim.shape}')
print(f'animation data: {other_anim.shape}')

hip channels: ['Xposition', 'Yposition', 'Zposition', 'Zrotation', 'Yrotation', 'Xrotation']
hip data: (344, 6)
animation data: (344, 31, 3)


In [5]:
# win = pyglet.window.Window(500,500,resizable=True)

# # do not increase z
# hip_anim[:,:2] = scale_range(hip_anim[:,:2], 0, 500)+150
# hip_anim[:,2] = scale_range(hip_anim[:,2], -1, 1)

# @win.event
# def on_draw():
#     glBegin(GL_POINTS)
#     for i in range(hip_anim.shape[0]):
#         x = hip_anim[i][0]
#         y = hip_anim[i][1]
#         z = hip_anim[i][2]
#         glVertex3f(x,y,z)
#     glEnd()

# pyglet.app.run()

# what are the steps? 

1 - calculate the position of the root point. Hip position.

2 - calculate the position for frame 1 of all the other joints based on this. that's why frame 1 is all 0s under MOTION, they joints are all at the default offsets and there's not a lot of rotation going on. The few taht are non zero dont do much set the arms to right angles, etc.

3 - build a function that can take a point and rotate it in space. we need this because for each frame our renderer will take the skeleton at its default offsets (the starting frame essentially) and then apply rotations recursively to get a new skeleton with joints positions.

4 - add the ability to draw lines for each joint.

# step 1 get the position of the hip

In [6]:
#just grabe the first 3 values
hip_anim[:,:3].shape

(344, 3)

# step 2 calculate the position of other joints

In [30]:
with open(file) as f:
    bvh_r = bvh.Bvh(f.read())
    j = bvh_r.get_joint('Hips')
    print(bvh_r.get_joint_index('Hips'))

0


In [22]:
# get the joint positions in absolute space
# recursively goes through all joints
# and finds their initial position, in 
# absolute spac
def build_positions(file):
    tree = []
    lines = []
    def check_children(node, parent, parent_offset):
        parent_offset = np.array([float(x) for x in parent_offset])
        child_offset = np.array([float(x) for x in node['OFFSET']])
        node_offset = parent_offset+child_offset
        tree.append((node, node_offset))
        if np.sum(node_offset) > 0:
            lines.append((parent, ))
        for child in node:
            if 'JOINT' in child.value:
                parent_offset = node['OFFSET']
                check_children(child, node, parent_offset)
            
    with open(file) as f:
        bvh_r = bvh.Bvh(f.read())
        j = bvh_r.get_joint('Hips')
        check_children(j, None, j['OFFSET'])
        
    return tree, lines
        
tree, lines = build_positions(file)
tree

[(ROOT Hips, array([0., 0., 0.])),
 (JOINT LHipJoint, array([0., 0., 0.])),
 (JOINT LeftUpLeg, array([ 1.65674, -1.80282,  0.62477])),
 (JOINT LeftLeg, array([ 4.25394, -8.93858,  0.62477])),
 (JOINT LeftFoot, array([  5.08956, -13.98346,   0.     ])),
 (JOINT LeftToeBase, array([ 2.6894 , -7.38906,  2.14581])),
 (JOINT RHipJoint, array([0., 0., 0.])),
 (JOINT RightUpLeg, array([-1.6107 , -1.80282,  0.62476])),
 (JOINT RightLeg, array([-4.20572, -8.93259,  0.62476])),
 (JOINT RightFoot, array([ -5.06282, -13.91001,   0.     ])),
 (JOINT RightToeBase, array([-2.69804, -7.41282,  2.13368])),
 (JOINT LowerBack, array([0., 0., 0.])),
 (JOINT Spine, array([ 0.01961,  2.0545 , -0.14112])),
 (JOINT Spine1, array([ 0.02982,  4.11886, -0.20033])),
 (JOINT Neck, array([ 0.01021,  2.06436, -0.05921])),
 (JOINT Neck1, array([0.00713, 1.56711, 0.14968])),
 (JOINT Head, array([0.04142, 3.12752, 0.04962])),
 (JOINT LeftShoulder, array([ 0.01021,  2.06436, -0.05921])),
 (JOINT LeftArm, array([ 3.54205

Now let's try to render this.

In [23]:
# lines = np.array([np.array([line[0],line[1]]) for line in lines])
# skel = np.array([joint[1] for joint in tree])
# win = pyglet.window.Window(500,500,resizable=True)

# # do not increase z
# skel[:,:2] = scale_range(skel[:,:2], 0, 500)+150
# skel[:,2] = scale_range(skel[:,2], -1, 1)

# lines[:,:,:2] = scale_range(lines[:,:,:2], 0, 500)+150
# lines[:,:,2] = scale_range(lines[:,:,2], -1, 1)

# @win.event
# def on_draw():
#     glBegin(GL_POINTS)
#     for i in range(skel.shape[0]):
#         x = skel[i][0]
#         y = skel[i][1]
#         z = skel[i][2]
#         glVertex3f(x,y,z)
#     glEnd()

# pyglet.app.run()

It renders something close to a crucifix of dots.

# step 3 rotate points.

The 6 values that the root joint has for all 344 frames of this animation are the values in channels. Let's print the position channels single frame of root.

See: https://www.siggraph.org/education/materials/HyperGraph/modeling/mod_tran/3drota.htm#Z
```
# rotation around z aka Zrotation
x' = x*cos q - y*sin q
y' = x*sin q + y*cos q 
z' = z

# rotation around y
z' = z*cos q - x*sin q
x' = z*sin q + x*cos q
y' = y

# rotation around x
y' = y*cos q - z*sin q
z' = y*sin q + z*cos q
x' = x
```

In [24]:
def rotate_point(point, angles):
    '''
    we could use rotation matrices but well stick to simple if statemeents
    as they basically do the same thing here (and we just avoid the matrix math)
    args
        point is a 3D np array with the x,y,z coordinates of the point
        angles are the euler angles in degrees of which rotations to perform. zrot, yrot, xrot
    returns
        a new position for this point as a np array
    '''
    x,y,z = point
    # rotate x,y,z position around all axes of rotation
    for i,rot in enumerate(angles):
        #rotate about z
        sin_rot = math.sin(rot)
        cos_rot = math.cos(rot)
        if i == 0:
            x = x*cos_rot - y*sin_rot
            y = x*sin_rot + y*cos_rot
        # rotate about y
        elif i == 1:
            x = z*sin_rot + x*cos_rot
            z = z*cos_rot - x*sin_rot
        # rotate about x
        elif i == 2:
            y = y*cos_rot - z*sin_rot
            z = y*sin_rot + z*cos_rot
    return [x,y,z]

assert rotate_point([10.4194, 16.7048, -30.1003], [0, 0, 0]) == [10.4194, 16.7048, -30.1003]
# probably need some extra assertions here!

In [46]:
hip_anim = get_joint_anim(file, 'Hips')
other_anim = get_anims(file)
positions, lines = build_positions(file)

In [47]:
hip_anim.shape, other_anim.shape

((344, 6), (344, 31, 3))

In [48]:
lines

[(JOINT LHipJoint, JOINT LeftUpLeg),
 (JOINT LowerBack, JOINT Spine),
 (JOINT Spine, JOINT Spine1),
 (JOINT Spine1, JOINT Neck),
 (JOINT Neck, JOINT Neck1),
 (JOINT Neck1, JOINT Head),
 (JOINT Spine1, JOINT LeftShoulder),
 (JOINT LeftShoulder, JOINT LeftArm),
 (JOINT LeftArm, JOINT LeftForeArm),
 (JOINT LeftForeArm, JOINT LeftHand),
 (JOINT LeftHand, JOINT LeftFingerBase),
 (JOINT LeftFingerBase, JOINT LeftHandIndex1),
 (JOINT LeftHand, JOINT LThumb),
 (JOINT Spine1, JOINT RightShoulder)]

In [52]:
def get_anim():
    anim_lines = []
    anim_positions = []
    for hip_frame, other_frame in zip(hip_anim, other_anim):
        # apply rotation to each point
        new_positions = []
        # get the relative position and add the position of the hip
        # this is the base position for the frame that we then rotate
        # to get the actual position
        hip_pos = hip_frame[:3]
        for i, (joint, rel_position) in enumerate(positions[1:]):
            new_positions.append(rotate_point(hip_pos+rel_position, other_frame[i]))
        hip_position = np.array(rotate_point(hip_pos, hip_frame[3:]))
        new_positions = np.array(new_positions)
        stacked_pos = np.vstack([hip_position, new_positions])
        anim_positions.append(stacked_pos)
        
        new_lines = []
        for line in lines:
            idx1 = bvh_r.get_joint_index(line[0].name)
            idx2 = bvh_r.get_joint_index(line[1].name)
            new_lines.append(np.array([stacked_pos[idx1], stacked_pos[idx2]]))
        
        anim_lines.append(np.array(new_lines))
        
    return np.array(anim_positions), np.array(anim_lines)

anim_points, anim_lines = get_anim()
anim_points.shape, anim_lines.shape

((344, 31, 3), (344, 14, 2, 3))

In [20]:
frame = None
def update_frame(a,b):
    global frame
    if frame == None or frame == anim_positions.shape[0]-1:
        frame = 0
    else:
        frame += 1

def animate_frames(data):
    win = pyglet.window.Window(500,500,resizable=True)
    
    # do not increase z
    data[:,:,:2] = scale_range(data[:,:,:2], 0, 500)+100
    data[:,:,2] = scale_range(data[:,:,2], -1, 1)
    
    @win.event
    def on_draw():
        glClear(GL_COLOR_BUFFER_BIT)
        glBegin(GL_POINTS)
        for i in range(data.shape[1]):
            x = data[frame][i][0]
            y = data[frame][i][1]
            z = data[frame][i][2]
            glVertex3f(x,y,z)
        glEnd()
    pyglet.clock.schedule(update_frame, 1/10.0)
    pyglet.app.run()
    
animate_frames(np.array(anim_positions))